In [1]:
import numpy as np
import argparse
import c3.libraries.fidelities as fidelities
from utils import *
import c3.libraries.algorithms as algorithms
import c3.utils.qt_utils as qt_utils
import c3.libraries.constants as constants
from multi_level_qubits.TwoQubitsExperiment import TwoQubitsExperiment

2021-10-20 15:53:10.818894: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-20 15:53:10.818926: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("output", help="Output directory")
args = parser.parse_args()
output_dir = args.output

In [3]:
def spectral_peaks_goal(propagators: dict, instructions: dict, index, dims,
                        qubit: chip.Qubit, generator: Generator, drive: chip.Drive,
                        n_eval=-1, num_peaks=2) -> tf.Tensor:
    # find energies of the qubit
    energies = qubit.get_Hamiltonian().numpy().diagonal().real
    vec1 = np.array([energies[3] - energies[2], energies[1] - energies[0]]) / (2 * np.pi)

    infids = []
    for gate, propagator in propagators.items():
        # create signal and find peaks
        instr = instructions[gate]
        signal = generator.generate_signals(instr)[drive.name]
        peakFrequencies, peakValues = findFrequencyPeaks(signal["ts"].numpy(), signal["values"].numpy(), num_peaks)
        peakFrequencies = tf.convert_to_tensor(peakFrequencies)
        peakValues = tf.convert_to_tensor(peakValues)

        # add frequency offsets to infidelity
        #infid = np.linalg.norm(vec1 - peakFrequencies)
        infid1 = tf.abs(vec1[0] - peakFrequencies[0]) / peakFrequencies[0]
        infid1 += tf.abs(vec1[1] - peakFrequencies[1]) / peakFrequencies[1]

        # add peak height difference to infidelity
        if peakFrequencies[1] > peakFrequencies[0]:
            a = peakValues[1]
            b = np.sqrt(2) * peakValues[0]
        else:
            a = peakValues[0]
            b = np.sqrt(2) * peakValues[1]
        infid2 = tf.abs(a - b) / tf.maximum(a, b)
        infid = infid1 * (1 + infid2)
        infids.append(infid)
    result = tf.reduce_mean(infids)
    print("type: ", type(result))
    return result


def printInformation(exper: TwoQubitsExperiment, name: str) -> None:
    signal = exper.generateSignal()
    peakFrequencies, peakValues = findFrequencyPeaks(signal["ts"].numpy(), signal["values"].numpy(), 4)
    print("peaks: ", np.sort(peakFrequencies))
    exper.saveSignal("signal_" + name)
    exper.plotSignal("signal_" + name)
    exper.saveTimeEvolution("populations_" + name)
    exper.plotTimeEvolution("populations_" + name)
    exper.savePropagator("propagator_" + name)
    exper.plotPropagator("propagator_" + name)

In [4]:
freq1 = 5e9
freq2 = 4e9
anharm = 300e6
t_final = 15e-9
sigma = t_final / 15
sigma2 = sigma / 2
relative_amp = 2
carrier_freq = 4.7e9
coupling_strength = 0 #20e6
optimisable_params = {
    "pwc": [
        "amp",
        "xy_angle", "freq_offset",
        "delta",
        "t_final", #"t_bin_start", "t_bin_end",
        "inphase"
    ],
    "carrier": [
        "freq",
        "framechange"
    ]
}

In [ ]:
ideal_Unity = qt_utils.np_kron_n([constants.Id, constants.Id])
ideal_X90_q1 = qt_utils.np_kron_n([constants.x90p, constants.Id])
ideal_X90_q2 = qt_utils.np_kron_n([constants.Id, constants.x90p])
ideal_CNOT_q1 = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]], dtype=np.complex128)
ideal_CNOT_q2 = constants.GATES["cx"]

In [5]:
envelope = createPWCDoubleGaussianPulse(t_final, sigma, sigma2, relative_amp, 30)

exper = TwoQubitsExperiment(directory=f"{output_dir}/")
exper.prepare(t_final, (freq1, freq2), (anharm, anharm), carrier_freq, envelope,
              ideal_gate=ideal_X90_q2,
              occupied_levels=[0,2],
              useDrag=True)

exper.saveIdealPropagator(name="ideal_propagator")

# print information
energies = exper.getEnergies()
print("energies: ", energies)
print("energies differences: ", [(energies[i + 1] - energies[i]) / (2 * np.pi) for i in range(len(energies) - 1)])
printInformation(exper, "before")

# first optimisation
callback = lambda infid: print(infid)
params_after = exper.optimise(
    optimisable_params,
    algorithm=algorithms.lbfgs,
    algorithm_options={
        #"stop_at_convergence": 15,
        #"spread": 0.2,
        #"popsize": 25,
        "maxfun": 200,
    },
    fidelity_function=fidelities.unitary_infid_set,
    fidelity_params={
        "active_levels": 4,
        #"qubit": exper.getQubit(),
        #"generator": exper.getExperiment().pmap.generator,
        #"drive": getDrive(exper.getExperiment().pmap.model, exper.getQubit())
        #"penalty_factor": 1
    },
    callback=callback,
)
printInformation(exper, "after")
print(params_after)

2021-10-20 15:53:41.634032: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-20 15:53:41.634239: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-20 15:53:41.634253: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-20 15:53:41.634273: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (localhost.localdomain): /proc/driver/nvidia/version does not exist
2021-10-20 15:53:41.634882: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


Dimensions:  25
enabling DRAG2


2021-10-20 15:53:48.644297: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-20 15:53:48.660615: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2793540000 Hz


energies:  [-2.79259937e+05  2.51296511e+10  4.83757632e+10  6.97374037e+10
  8.92159401e+10  3.14178611e+10  5.65444429e+10  7.97898310e+10
  1.01151296e+11  1.20625834e+11  6.09531732e+10  8.60696421e+10
  1.09315784e+11  1.30678041e+11  1.50147518e+11  8.86104349e+10
  1.13655472e+11  1.36944365e+11  1.58314016e+11  1.77778209e+11
  1.14444010e+11  1.39537488e+11  1.62779834e+11  1.84141307e+11
  2.03580790e+11]
energies differences:  [3999552640.2391567, 3699733644.580575, 3399810676.420923, 3100105355.5315375, -9198849979.099773, 3999019701.598215, 3699618425.235013, 3399782719.982929, 3099469073.674329, -9497199005.288082, 3997410185.5457864, 3699738428.256548, 3399908804.4514093, 3098663525.75392, -9793931027.118484, 3986041502.0616775, 3706542412.304876, 3401085506.6842456, 3097822562.231657, -10079950811.207684, 3993751094.1205506, 3699134195.3207116, 3399784101.643265, 3093889682.32495]
C3:STATUS:Saving as: /home/user/c3/output_0/log/c1_unitary_infid_set_lbfgs/2021_10_20_T_15

FileNotFoundError: [Errno 2] No such file or directory: '/home/user/c3/output_0/log/c1_unitary_infid_set_lbfgs/2021_10_20_T_15_53_49/best_point_open_loop.log'